In [ ]:
import praw
from datetime import datetime
import pandas as pd
import requests
import time
comment_counts = {}
#2/2/20202 00:00:00 1612242000
current_time = 1612242000
#12/1/20202 00:00:00 1606798800
while(current_time > 1606798800):
    #Moving the query back by 5 mintues
    url = f'https://api.pushshift.io/reddit/comment/search/?after={current_time-300}&before={current_time}&subreddit=wallstreetbets&q=gme&size=100&score=%3E5'
    try:
        res = requests.get(url).json()
        date = datetime.fromtimestamp(res['data'][0]['created_utc']).strftime('%m/%d/%Y %H:00')
        time.sleep(0.3)
    except:
        current_time -= 300
        continue
    if date in comment_counts:
        comment_counts[date] += len(res['data'])
    else:
        comment_counts[date] = len(res['data'])
    print(date)
    current_time -= 300


dates = []
comments = []
for k,v in comment_counts.items():
    dates.append(k)
    comments.append(v) 
#Create a dataframe from the data taken from reddit 
df = pd.DataFrame({'Date': dates, '# of Comments': comments})
df.to_csv('12_1_to_2_2_hourly.csv')



In [ ]:
##################################################################################################

In [ ]:
from config import alphavantage_key as api_key
import requests
#This bring in the hourly trade data
for i in range(1,6):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=GME&interval=60min&slice=year1month{i}&apikey={api_key}'
    #this only returns a CSV
    res = requests.get(url)
    #download the returned CSV 
    open(f'../Data/{i}_months_before_.csv', 'wb').write(res.content)

In [ ]:
##################################################################################################

In [ ]:
#Did not use this information -- too limited in the data we received, only 100 submissions returned
#Working with the python reddit api wrapper
import praw
from config import client_id, secret
from datetime import datetime
import pandas as pd

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=secret,
    user_agent="ckioski"
)
subreddit = reddit.subreddit('wallstreetbets')
comment_counts = {}
for submission in subreddit.search(query='GME',sort='comments',time_filter='year',limit=None):
    print(1)
    #unix time for Feb 3rd as 12:00 am.  We are only looking at posts from before that 
    if submission.created_utc > 1612328400:
        continue
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        try:
            print(comment.created_utc)
            if 'GME' in comment.body:
                date = datetime.fromtimestamp(comment.created_utc).strftime('%m/%d/%Y %H:00')
                if date in comment_counts:
                    comment_counts[date] += 1
                else:
                    comment_counts[date] = 1
        except:
            pass
dates = []
comments = []
for k,v in comment_counts.items():
    dates.append(k)
    comments.append(v) 
#Create a dataframe from the data taken from reddit 
df = pd.DataFrame({'Date': dates, '# of Comments': comments})
df.to_csv('Data/comment_results.csv')


